In [1]:
import pandas as pd

In [2]:
invoice_data = pd.read_excel('inline_data.xlsx', sheet_name = 'invoice_data')
bank_statement = pd.read_excel('inline_data.xlsx', sheet_name = 'bank_statement')
buyer_info = pd.read_excel('inline_data.xlsx', sheet_name = 'buyer_info')
invoice_data.head()

,買方統一編號,買方名稱,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,總開立發票數,總開立金額
0,72821126,星X國際有限公司板橋店,1800,1800,1800,1800,1800,0,5,9000
1,54345001,老X香實業有限公司,4503,0,0,3437,0,3892,3,11832
2,50906000,慕X諾國際股份有限公司,0,0,0,0,3600,3600,4,7200


In [3]:
bank_statement.head()

,買方帳號,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,總筆數,總存入金額
0,00288818****9671,0,1800,1800,1800,1800,1800,5,9000
1,00288818****9672,0,0,4503,0,3437,0,2,7940


In [4]:
buyer_info.head()

,買方名稱,買方統一編號,買方帳號,買方信箱
0,星X國際有限公司板橋店,72821126,00288818****9671,chopper870907@gmail.com
1,老X香實業有限公司,54345001,00288818****9672,chopper870907@gmail.com
2,慕X諾國際股份有限公司,50906000,00288818****9673,chopper870907@gmail.com


In [5]:
unpaid = list()
total_invoice = list()
unpaid_rate = list()
for i in buyer_info.index:
    invoice = 0
    deposit = 0
    for j in invoice_data.index:
        if buyer_info['買方統一編號'][i] == invoice_data['買方統一編號'][j]:
            invoice = invoice_data['總開立金額'][j]
    for k in bank_statement.index:
        if buyer_info['買方帳號'][i] == bank_statement['買方帳號'][k]:
            deposit = bank_statement['總存入金額'][k]
    unpaid.append(invoice - deposit)
    total_invoice.append(invoice)
    unpaid_rate.append('{:.2%}'.format((invoice - deposit) / invoice)) 

In [6]:
buyer_info['未付總金額'] = unpaid
buyer_info['總開立金額'] = total_invoice
buyer_info['欠款比例'] = unpaid_rate
buyer_info.head()

,買方名稱,買方統一編號,買方帳號,買方信箱,未付總金額,總開立金額,欠款比例
0,星X國際有限公司板橋店,72821126,00288818****9671,chopper870907@gmail.com,0,9000,0.00%
1,老X香實業有限公司,54345001,00288818****9672,chopper870907@gmail.com,3892,11832,32.89%
2,慕X諾國際股份有限公司,50906000,00288818****9673,chopper870907@gmail.com,7200,7200,100.00%


In [7]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

In [8]:
for i in buyer_info.index:
    if buyer_info['未付總金額'][i] > 0:
        mail = MIMEMultipart()  #建立MIMEMultipart物件
        mail["subject"] = "Inline payment notification"  #郵件標題
        mail["from"] = ""  #寄件者
        mail["to"] = str(buyer_info['買方信箱'][i]) #收件者
        msg = "Dear " + str(buyer_info["買方名稱"][i]) + "," \
        + "\n\nYour accumulated payment is $" + str(buyer_info["未付總金額"][i]) + "." \
        + "\nPlease kindly remit the payment as soon as possible." \
        + "\nWe appreciate a lot for your cooperation. " \
        + "\n\nBest regards, \nInline"
        mail.attach(MIMEText(msg))  #郵件內容
        try:
            smtp = smtplib.SMTP(host="smtp.gmail.com", port="587")  # 設定SMTP伺服器
            smtp.ehlo()  # 驗證SMTP伺服器
            smtp.starttls()  # 建立加密傳輸
            smtp.login("", "")  # 登入寄件者gmail
            smtp.send_message(mail)  # 寄送郵件
            print("Complete!")
        except Exception as e:
            print("Error message: ", e)

Complete!
Complete!
